# 模型构造

## 层和块

In [1]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.1205, -0.0558, -0.0393, -0.2034,  0.0516,  0.0426,  0.0419,  0.0786,
         -0.0591,  0.2674],
        [-0.1874, -0.2384, -0.0157, -0.1506,  0.0914, -0.1008,  0.1284,  0.0912,
          0.0504,  0.2754]], grad_fn=<AddmmBackward0>)

`nn.Sequential`定义了一种特殊的`Module`

**自定义块**

In [2]:
class MLP(nn.Module):
    """是nn.Module的一个子类，有两个比较重要的函数 __init__ 和 forward"""
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.out = nn.Linear(256, 10)
        
    # 前向函数
    def forward(self, X):
        return self.out(F.relu(self.hidden(X)))

实例化多层感知机的层，然后在每次调用正向传播函数时调用这些层

In [3]:
net = MLP()
net(X)

tensor([[ 0.0622, -0.1287,  0.2647, -0.1168, -0.0008,  0.1092,  0.0821,  0.2337,
         -0.1064, -0.0867],
        [ 0.1724, -0.1707,  0.2765,  0.0167,  0.0119,  0.1952,  0.0535,  0.1861,
         -0.1621, -0.1435]], grad_fn=<AddmmBackward0>)

**顺序块**

In [6]:
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            self._modules[block] = block # 特殊容器，ordering dictionary
            
    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2301,  0.0168, -0.0499,  0.0095, -0.0565, -0.0409,  0.1120,  0.1876,
         -0.0978,  0.0661],
        [-0.2449,  0.0672, -0.1641, -0.0047, -0.1542,  0.2708,  0.0646,  0.0710,
          0.0145, -0.0342]], grad_fn=<AddmmBackward0>)

在正向传播函数中执行代码

In [7]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)
    

tensor(0.3437, grad_fn=<SumBackward0>)